In [1]:
import pandas as pd

#Import Pandas and find load our datafile
censusProfile = pd.read_csv('Data/Proximity.csv')
censusProfile = censusProfile[censusProfile['PRNAME'].isin(['Ontario'])]
censusProfile = censusProfile.sort_values(by=['CSDUID'])
censusProfile.to_csv('output/ontarioprox.csv')

censusNames = censusProfile['CSDUID'].unique()


In [2]:
groupedCensus = censusProfile.groupby('CSDUID')
csdIndexed = {}
categories = ['prox_idx_emp', 'prox_idx_pharma', 'prox_idx_childcare',
              'prox_idx_health', 'prox_idx_grocery', 'prox_idx_educpri',
              'prox_idx_educsec','prox_idx_lib','prox_idx_parks','prox_idx_transit']
for censusID in censusNames:
    singleCSDProximities = groupedCensus.get_group(censusID)
    singleCSDProximities = singleCSDProximities.reset_index()
    csdIndexed[censusID] = 0.0
    i = 0
    disseminationAggregate = []
    #For each dissemination block in the census subdivision, aggregate its proximity index of the 10 indices
    for index, row in singleCSDProximities.iterrows():
        disseminationAggregate.append(0)
        for cat in categories:
            if (row[cat] == '' or row[cat] == '..' or row[cat] == 'F'):
                row[cat] = 0
            disseminationAggregate[i] = disseminationAggregate[i] + (float)(row[cat])
        disseminationAggregate[i] = disseminationAggregate[i]/10
        i = i + 1
    #add up all the dissemination blocks and divide by the number of dissemination blocks in the census subdivision
    #this will give us an aggergate of the proximity index for the census subdivision
    for db in disseminationAggregate:
        csdIndexed[censusID] = csdIndexed[censusID] + db
    csdIndexed[censusID] = csdIndexed[censusID] / (i)

    

In [68]:
outputDataset = pd.DataFrame(columns=['CSDUID', 'Proximity_Index'])

datasetIndex = 0

for censusID in censusNames:
    outputDataset.loc[datasetIndex] = [(int)(censusID)] + [csdIndexed[censusID]]
    datasetIndex = datasetIndex + 1
        
print(outputDataset)
outputDataset.to_csv('output/proximityIndexbyCSD.csv')


        CSDUID  Proximity_Index
0    3501005.0         0.004466
1    3501007.0         0.000379
2    3501011.0         0.004356
3    3501012.0         0.032407
4    3501020.0         0.011553
..         ...              ...
570  3560097.0         0.000877
571  3560098.0         0.000000
572  3560100.0         0.000000
573  3560102.0         0.000000
574  3560104.0         0.000260

[575 rows x 2 columns]
